# 하이퍼파라미터 최적화 후 재평가할 필요가 있다

- 예시
    1. 좋은 하이퍼파라미터를 찾음 -> 결과를 분석하기 위해 가장 좋은 하이퍼파라미터로 목적함수 실행
    2. Train 시간을 줄이기 위해 부분 데이터 세트로 하이퍼파라미터 값을 찾은 뒤 전체 데이터세트를 이용할 경우
    
- `best_trial`은 가장 좋은 하이퍼파라미토로 목적함수를 재평가하는 데 쓸 수 있다.
- 이 예제는 예시 1.의 케이스를 따라감

In [2]:
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import optuna

def objective(trial):
    X, y = make_classification(n_features = 10, random_state = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    
    C = trial.suggest_float('C', 1e-7, 10.0, log = True)
    
    clf = LogisticRegression(C = C)
    clf.fit(X_train, y_train)
    
    return clf.score(X_test, y_test)

study = optuna.create_study(direction = 'maximize')
study.optimize(objective, n_trials = 10)

print(study.best_trial.value)

[I 2022-12-12 16:29:54,378] A new study created in memory with name: no-name-992c27b0-4fb8-429a-94ca-5e19a936df00
[I 2022-12-12 16:29:54,396] Trial 0 finished with value: 0.44 and parameters: {'C': 3.850703502055514e-07}. Best is trial 0 with value: 0.44.
[I 2022-12-12 16:29:54,407] Trial 1 finished with value: 0.44 and parameters: {'C': 0.00011418652512111388}. Best is trial 0 with value: 0.44.
[I 2022-12-12 16:29:54,419] Trial 2 finished with value: 0.92 and parameters: {'C': 0.9328850130551871}. Best is trial 2 with value: 0.92.
[I 2022-12-12 16:29:54,429] Trial 3 finished with value: 0.92 and parameters: {'C': 0.7834705892070642}. Best is trial 2 with value: 0.92.
[I 2022-12-12 16:29:54,442] Trial 4 finished with value: 0.88 and parameters: {'C': 0.006448574593019414}. Best is trial 2 with value: 0.92.
[I 2022-12-12 16:29:54,460] Trial 5 finished with value: 0.88 and parameters: {'C': 0.008282547274306785}. Best is trial 2 with value: 0.92.
[I 2022-12-12 16:29:54,478] Trial 6 finis

0.92


HPO 후, 같은 데이터세트에서 Recall, Precision, f1-score 등의 평가 지표를 계산한다고 가정한다. 이 때, 대부분의 요소를 공유하는 **또다른 목적함수를 정의**하여 모델을 재현할 수 있다.

In [3]:
def detailed_objective(trial):
    X, y = make_classification(n_features = 10, random_state = 1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)
    
    C = trial.suggest_float('C', 1e-7, 10.0, log = True)
    
    clf = LogisticRegression(C = C)
    clf.fit(X_train, y_train)
    
    # 다른 지표들 계산하기
    pred = clf.predict(X_test)
    
    acc = metrics.accuracy_score(pred, y_test)
    recall = metrics.recall_score(pred, y_test)
    precision = metrics.precision_score(pred, y_test)
    f1 = metrics.f1_score(pred, y_test)
    
    return acc, f1, recall, precision

In [4]:
detailed_objective(study.best_trial) # acc, f1, recall, precision

(0.92, 0.9285714285714286, 0.9285714285714286, 0.9285714285714286)

## `best_trial`과 `Ordinal Trials`의 차이점
- `best_trial`은 `FrozenTrial`을 반환한다 : `FrozenTrial`은 특정 상황에선 `Trial`과 다르게 반응한다.
    - 예를 들면 Pruning은 `should_prune`이 항상 False이기 때문에 작동하지 않음
- 다목적함수의 경우, `best_trials`(s가 붙음) Pareto Front에서의 `FrozenTrial`의 리스트를 반환한다